# Linear Regression
### 회귀분석 모형

다음 특징 변수 들을 이용하여 테스트 데이터에 예상되는 주택 가격을 도출 하시오.

예측 값과 실제 값의 차이 (오차율)의 합을 평균 낸 점수로 평가한다.

In [64]:
# 데이터 로드
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

df = pd.read_csv('./extrafiles/house_price.csv')
df

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000
...,...,...,...,...,...,...
17684,34,2.3013,0.214583,2.748299,4.897959,26600
17685,33,2.6750,0.246622,3.428571,4.698413,22500
17686,39,2.3667,0.340771,1.876812,3.572464,17500
17687,19,2.1000,0.386107,2.987805,3.774390,14999


In [65]:
# 결측값 & 이상치 확인
pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 2000)

#  다중공선성 확인
print(df.corr())

# bedroom 과 room 이 선형 관계를 이루고 있음
# 주택 가격을 예측하는 문제에서 room 특징 변수 하나면 충분한듯.

# df.drop(['bedrooms', 'income'], axis=1 , inplace=True)
df.columns

             housing_age    income  bedrooms  households     rooms  house_value
housing_age     1.000000 -0.187740  0.143249    0.078840 -0.251983     0.025387
income         -0.187740  1.000000 -0.666467   -0.061368  0.612958     0.661947
bedrooms        0.143249 -0.666467  1.000000    0.013780 -0.812933    -0.241313
households      0.078840 -0.061368  0.013780    1.000000 -0.056004    -0.245060
rooms          -0.251983  0.612958 -0.812933   -0.056004  1.000000     0.243877
house_value     0.025387  0.661947 -0.241313   -0.245060  0.243877     1.000000


Index(['housing_age', 'income', 'bedrooms', 'households', 'rooms', 'house_value'], dtype='object')

In [66]:
# x, y 분리
X = df[['housing_age', 'income', 'households', 'rooms']]
y  = df[['house_value']]

X.shape
# (17689, 3)

from sklearn.preprocessing import MinMaxScaler
X_scaled = MinMaxScaler().fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, index=X.index, columns=X.columns)

# 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=333)


In [73]:
# 모델 적용
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X_train, y_train)
pred_train = model.predict(X_train)
print("예측 값 들 : >",  pred_train)
print("설명력(R2) : ", model.score(X_train, y_train))

# model score가 리턴 하는 값은 설명력(R-square) 이다.
from sklearn.metrics import accuracy_score

# 설명력이 잘 예측한 정도라면 오차율을 통해 모델의 성능 평가를 할 수 있다.
# RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, pred_train) # MSE(평균제곱오차) 는 오차값의 + - 를 양의 부호로 변경해주는 역할을 한다.

print("MSE is : >>", mse)
print("RMSE is : >> ", np.sqrt(mse)) # 제곱근은 너무 커진 값을 원래 스케일로 돌려주는 역할을 한다.
# 실제값과 예측값간 차이 - 평균적으로 64700 달러 정도의 오차가 존재한다.

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_train, pred_train)
print("MAE is : >>", mae)

# MAPE
def MAPE(y, pred):
    return np.mean(np.abs(y - pred) / y)*100
print(MAPE(y_train, pred_train))

# 평균적으로 실제 주택 가격과 예측 가격 사이에 32% 정도의 오차율을 보이고 있다.

예측 값 들 : > [[ 79770.25884643]
 [103998.35027797]
 [105290.60653913]
 ...
 [218396.40727883]
 [331687.96923188]
 [125583.08660264]]
설명력(R2) :  0.543497882641496
MSE is : >> 4195771304.364143
RMSE is : >>  64774.77367281296
MAE is : >> 48836.58058846101
house_value    32.25042
dtype: float64


In [68]:
y_comp = y_train.reset_index()

data = pd.concat([y_comp, pd.DataFrame(pred_train)], axis=1)
data

,index,house_value,0
0,17352,54400,79770.258846
1,11245,142800,103998.350278
2,12082,132400,105290.606539
3,12610,124200,262496.552850
4,6537,204800,232860.371469
...,...,...,...
13261,8477,175900,190977.269774
13262,3516,265700,277083.261790
13263,5494,223500,218396.407279
13264,973,369200,331687.969232


In [72]:
data[data['house_value'] == data[0]]

,index,house_value,0


In [ ]:
# r-square 는 일치하는 값의 비율이 아니다.
# R-Square 공식

# (전체 편차들의 제곱 합 - 전체 잔차들의 제곱 합) / 전체 편차들의 제곱 합